# SVM - Climate Sentiment Multiclass Classification
## CS522 Project

Elliptical Envelope Denoise

### Dataset: 
https://www.kaggle.com/code/luiskalckstein/climate-sentiment-multiclass-classification

### Imports

In [1]:
import numpy as np
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.preprocessing import MultiLabelBinarizer
from sklearn.multiclass import OneVsRestClassifier
from sklearn.svm import LinearSVC
from sklearn.calibration import CalibratedClassifierCV
from Common.DataCenter import data_center
from Common.preprocessor import normalize_preprocessing
from Common.UtilFuncs import print_evaluation, Evaluator
from Common.EllipticalEnvelopeMethod import EllipticalEnvelopeMethod

%matplotlib inline

Ev  = Evaluator()


### Text preprocessing

In [2]:
# parameter: original X of training set and test set
# return:  vectorised X of training set and test set
def text_preprocessing(X_train, X_test):
    # preprocessing with traditional NLP methodology
    X_train_normalized = normalize_preprocessing(X_train)
    X_test_normalized  = normalize_preprocessing(X_test)
    
    # vectorization
    # Convert texts to vectors by TFIDF
    vectorizer = TfidfVectorizer(ngram_range=(1,2))
    X_train_vec  = vectorizer.fit_transform(X_train_normalized)
    X_test_vec   = vectorizer.transform(X_test_normalized)
      
    return X_train_vec, X_test_vec

### One-hot encoding, convert the labels to vectors (4 x 1) each

In [3]:
# parameter: original y of training set, original y of test set
# return:  encoded y of training set and test set
def one_hot_encoding(y_train, y_test):
    mlb          = MultiLabelBinarizer()
    y_train_vec  = mlb.fit_transform(map(str, y_train))
    y_test_vec   = mlb.transform(map(str, y_test))
    return y_train_vec, y_test_vec


### Run SVM

In [4]:
# parameter:  vectorised X and encoded y of training set and test set
def run_SVM(X_train_vec, y_train_vec, X_test_vec, y_test_vec):
    # Run SVM - fit and predict
    SVM             = OneVsRestClassifier(LinearSVC(dual=False, class_weight='balanced'), n_jobs=-1)
    SVM.fit(X_train_vec, y_train_vec)
    y_pred          = SVM.predict(X_test_vec)
    return  y_pred


### Do an experiment

In [5]:
# Parameter: original X,y of training set and test set
def do_experiment(X_train, y_train, X_test, y_test):
    # Convert texts to vectors
    X_train_vec, X_test_vec = text_preprocessing(X_train, X_test)
    y_train_vec, y_test_vec = one_hot_encoding(y_train, y_test)

    # Run SVM and evaluate the results
    y_pred = run_SVM(X_train_vec, y_train_vec, X_test_vec, y_test_vec)

    # Print the evaluation
    print_evaluation(y_test_vec, y_pred, labels=[0,1,2,3])
    evaluateDF = Ev.evaluate(y_test_vec, y_pred)
    return evaluateDF

# do an experiment with denoise
# Parameter: original X,y of training set and test set
def do_experiment_denoise( trainDF, X_train, y_train, X_test, y_test, dimension):
    # Convert texts to vectors
    X_train_vec, X_test_vec = text_preprocessing(X_train, X_test)
    y_train_vec, y_test_vec = one_hot_encoding(y_train, y_test)

    # LearningWithNoisyLabels require the classifier has the entry predict_proba()
    # So, use CalibratedClassifierCV to wrap LinearSVC
    #SVM = CalibratedClassifierCV(LinearSVC(dual=False, class_weight='balanced'))
    SVM             = OneVsRestClassifier(LinearSVC(dual=False, class_weight='balanced'), n_jobs=-1)
    
    # clean noise in noisy data frame
    denoiseMethod = EllipticalEnvelopeMethod(dimension)
    cleanTrainDF, clean_X_train_vec, clean_y_train = denoiseMethod.CleanDFNoise(trainDF, X_train_vec, y_train )
    
    SVM.fit(clean_X_train_vec, clean_y_train)
    y_pred          = SVM.predict(X_test_vec)
    

    # Print the evaluation
    # One hot encoding for print_evaluation()
    _, y_pred = one_hot_encoding(y_train, y_pred)
    print_evaluation(y_test_vec, y_pred, labels=[0,1,2,3])

    evaluateDF = Ev.evaluate(y_test_vec, y_pred)
    return evaluateDF


### Main entry
**The settings of the noise sources.**

In [6]:
#Each entry: source type and (size, distribution)
noisy_set_sizes = {
    'mislabeled' : (8000, None),                   # max size: 15000
    'irrelevant' : (2000, [0.25,0.25,0.25,0.25]),  # max size: 34259
    'translated' : (2000, "reserve_labels"),       # max size: 5000
}


**Load the database and split it into training set, test set, noisy set, validation set**

In [7]:
# dc = data_center("twitter_sentiment_data_clean.csv", train_size = 20000, test_size = 4000, validation_size = 1000,
#                  noisy_size = noisy_set_sizes['mislabeled'][0] if 'mislabeled' in noisy_set_sizes.keys() else 0)

# Load the database and split it into training set, test set, noisy set, validation set
# Using noisy_set_sizes directly
dc = data_center("twitter_sentiment_data_clean.csv", train_size = 20000, test_size = 4000, validation_size = 1000,
                 noisy_size = noisy_set_sizes)


**Show the summary of the whole data**

In [8]:
dc.print_summary()


###################################### Data Summary #############################################
  Raw set (not cleaned) size: 40908
  Original set size: 40908
      sentiments ('Anti', 'Neutral', 'Pro', 'News'): 9.4%, 18.3%, 50.2%, 22.1%
  Training set size: 20000
  Test set size: 4000
  Noisy set size: 12000
  Validation set size: 1000
      noise sources ('mislabeled', 'irrelevant', 'translated'): 66.7%, 16.7%, 16.7%
##################################################################################################


**To see the data features via a demo**

In [9]:
train_df = dc.get_train_with_noisy_df(1000,1000)
data_center.print_data(train_df.head(15))


,noise,noise_text,sentiment,origin(sentiment),tweetid(partial),message(partial)
1120,1,mislabeled,3,2,794664,RT @ChelseaClinton: Thank you
34005,0,none,3,-1,605804,Obama to follow Al Gore into c
144,2,irrelevant,1,-1,-1,Can't wait to go on a field tr
29275,0,none,2,-1,951653,umm ??? so itâ€™s basically su
15593,1,mislabeled,0,2,848920,RT @Colorlines: POC who live t
3724,1,mislabeled,1,3,798878,RT @ReutersLive: LIVE: U.S. Se
11870,0,none,1,-1,840241,@bradcarlson_ not denying clim
1896,0,none,2,-1,796429,RT @emmaroller: A climate chan
120,0,none,1,-1,793214,@michellemalkin Trump wins ele
80,0,none,3,-1,793175,Satellites help scientists see


**Get the test set for evaluation.**

In [10]:
X_test, y_test = dc.get_test()


**Set distributions of training set.**

In [11]:
# distribution of training set
train_distribution = None


**Run experiments with different training sets, and use the same test set.**
**Please wait some minutes until the finish message appears.**

In [ ]:
expriment_no    = 1
print("-------------- No noisy training sets ----------")
for size in [2000, 4000, 5000, 8000, 10000, 15000, 20000]:
    # Get a training set without noisy data
    X_train, y_train = dc.get_train(size, train_distribution)

    print("* Training set size: %d samples" % (len(y_train)))
    data_center.print_distribution("  Sentiments", y_train)

    # Do an experiment
    dfResult = do_experiment(X_train, y_train, X_test, y_test)
    Ev.add_evaluation(dfResult, size, 0, "-",
                              data_center.calc_distribution_str(y_train, 'sentiment', [0,1,2,3]),
                              "-", expriment_no
                              )
    expriment_no += 1

print("-------------- Noisy training sets -------------")
# The noise source distribution of the whole noisy data.
dc.print_noise_source_distribution("General noise source distribution")

# here is the hyperparameter for dimension reduction
svdDimension = (400, 800, 1500)
dimensionIndex = 0
lstSizes = [(4000, 1000), (8000, 2000), (15000, 5000)]

for size in lstSizes:
    # Get a noisy training set
    train_df         = dc.get_train_with_noisy_df(size[0], size[1], train_distribution)
    # 在这里重置index
    train_df.reset_index(drop=True, inplace=True)
    X_train, y_train = data_center.Xy(train_df)
    X_noisy          = train_df[train_df['noise'] != 0]

    print("* Noisy training set size: %d samples (%d original, %d noisy)" % (len(y_train), size[0], size[1]))
    data_center.print_distribution("  Sentiments", y_train)
    dc.print_noise_source_distribution("  Noise sources")

    # Do an experiment without de-noising
    print("  Before de-noising:")
    dfResult = do_experiment(X_train, y_train, X_test, y_test)
    Ev.add_evaluation(dfResult, size[0], size[1], "N",
                            data_center.calc_distribution_str(y_train, 'sentiment', [0,1,2,3]),
                            data_center.calc_distribution_str(X_noisy, 'noise', [1,2,3]),
                            expriment_no
                            )

    # Do an experiment with de-noising
    print("  After de-noising:")
    dfResult = do_experiment_denoise(train_df, X_train, y_train, X_test, y_test, svdDimension[dimensionIndex])
    dimensionIndex += 1
    Ev.add_evaluation(dfResult, size[0], size[1], "Y",
                            data_center.calc_distribution_str(y_train, 'sentiment', [0,1,2,3]),
                            data_center.calc_distribution_str(X_noisy, 'noise', [1,2,3]),
                            expriment_no + len(lstSizes)
                            )
    expriment_no += 1
    
print("-------------- Experiments are finished. -------------")    


-------------- No noisy training sets ----------
* Training set size: 2000 samples
  Sentiments ('Anti', 'Neutral', 'Pro', 'News'): 9.4%, 18.3%, 50.2%, 22.1%
    f1 of classes: [0.349, 0.315, 0.711, 0.678]
    micro_f1: 0.626 , macro_f1: 0.513 , weighted_f1: 0.597, macro_precision: 0.592, macro_recall: 0.504
* Training set size: 4000 samples
  Sentiments ('Anti', 'Neutral', 'Pro', 'News'): 9.4%, 18.3%, 50.2%, 22.1%
    f1 of classes: [0.442, 0.371, 0.741, 0.695]
    micro_f1: 0.657 , macro_f1: 0.562 , weighted_f1: 0.635, macro_precision: 0.623, macro_recall: 0.554
* Training set size: 5000 samples
  Sentiments ('Anti', 'Neutral', 'Pro', 'News'): 9.4%, 18.3%, 50.2%, 22.1%
    f1 of classes: [0.466, 0.4, 0.746, 0.709]
    micro_f1: 0.668 , macro_f1: 0.580 , weighted_f1: 0.648, macro_precision: 0.636, macro_recall: 0.570
* Training set size: 8000 samples
  Sentiments ('Anti', 'Neutral', 'Pro', 'News'): 9.4%, 18.3%, 50.2%, 22.1%
    f1 of classes: [0.521, 0.42, 0.758, 0.719]
    micro_f1: 

/Users/guopei/miniforge3/envs/tf2/lib/python3.9/site-packages/sklearn/covariance/_robust_covariance.py:183: RuntimeWarning: Determinant has increased; this should not happen: log(det) > log(previous_det) (-7334.055974844926823 > -7448.981121492077364). You may want to try with a higher value of support_fraction (current value: 0.800).
  warnings.warn(
/Users/guopei/miniforge3/envs/tf2/lib/python3.9/site-packages/sklearn/covariance/_robust_covariance.py:183: RuntimeWarning: Determinant has increased; this should not happen: log(det) > log(previous_det) (-7338.086250161711177 > -7454.836621712873239). You may want to try with a higher value of support_fraction (current value: 0.800).
  warnings.warn(
/Users/guopei/miniforge3/envs/tf2/lib/python3.9/site-packages/sklearn/covariance/_robust_covariance.py:183: RuntimeWarning: Determinant has increased; this should not happen: log(det) > log(previous_det) (-7338.479698386255222 > -7343.936977397402188). You may want to try with a higher value

**Show evaluations in a form (be sure the experiments have been finished)**

In [ ]:
Ev.print()


**Parameters for plotting.**

In [ ]:
# Plot training set size vs. Macro F1
# x coordinate
xValue  = "x['Origin']+x['Noise']"
# also can try other numeric columns, like:
# xValue  = "x['Origin']"

# y coordinate
yValue  = "y['Macro F1']"
# also can try other numeric columns, like:
# yValue  = "y['Weighted F1']"
# yValue  = "y['Macro Precision']"

# Divide experiments into several groups, which will be plot as lines
lines = { # each item: name, filter
    'Original Data':      "df['Denoised']=='-'",
    'Noisy Data':       "df['Denoised']=='N'",
    'Denoised Data':    "df['Denoised']=='Y'",
}


**Plot the evaluations.**

In [ ]:
Ev.plot(xValue = xValue, yValue = yValue, lines = lines,
        title = 'SVM using Elliptic Envelope for de-noising')


In [ ]:

Ev.evaluateDF.to_clipboard(excel=True)